In [53]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn import metrics, preprocessing

In [54]:
os.chdir(r"D:\python_ML\python_ml_basic\data")

In [114]:
df = pd.read_csv('data_activity_training.csv', header='infer',encoding='latin1')
# print(df)

D:\python_ML\venv\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,14,19,22,25,70,73,86,87,89,90,94,97,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
# 결측치 3퍼 이상인것 제거
isnotZero = np.array(pd.isnull(df).mean(axis=0) <0.97)
df = df.loc[:,isnotZero]
# print(df)

In [119]:
# 특정 컬럼 제거
headerT = df.filter(regex='time',axis=1).columns
df2 = df.drop(columns=headerT)
df2 = df2.drop(columns=['Unnamed: 0','user_name','new_window'])
print(df2)

       num_window  roll_belt  pitch_belt  yaw_belt  total_accel_belt  \
0              11       1.41        8.07     -94.4                 3   
1              11       1.41        8.07     -94.4                 3   
2              11       1.42        8.07     -94.4                 3   
3              12       1.48        8.05     -94.4                 3   
4              12       1.48        8.07     -94.4                 3   
5              12       1.45        8.06     -94.4                 3   
6              12       1.42        8.09     -94.4                 3   
7              12       1.42        8.13     -94.4                 3   
8              12       1.43        8.16     -94.4                 3   
9              12       1.45        8.17     -94.4                 3   
10             12       1.45        8.18     -94.4                 3   
11             12       1.43        8.18     -94.4                 3   
12             12       1.42        8.20     -94.4              

In [11]:
#X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns

KeyError: "['is_spam'] not found in axis"

'yes', 'no' 레이블을 숫자로 변환:

In [20]:
LE = preprocessing.LabelEncoder()
Y = LE.fit_transform(Y)
Y

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [21]:
table = np.unique(Y, return_counts=True)
table

(array([0, 1], dtype=int64), array([736, 264], dtype=int64))

NaN이 있으면 채워 넣음:

In [22]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

#### 데이터 전처리:

In [23]:
X = preprocessing.scale(X)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3)

#### MLP  신경망 적용 (교차검증 최적화 포함):

In [25]:
hidden_layer_config = [30, 30, 30]
alpha_grid = [0.01, 0.1, 0.2, 0.5, 1.0, 2.0]
parameters = {'alpha': alpha_grid}

In [26]:
MLP = MLPClassifier(solver='lbfgs',hidden_layer_sizes=hidden_layer_config)
gridCV = GridSearchCV(MLP, parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_alpha = gridCV.best_params_['alpha']

In [13]:
print("MLP best alpha : " + str(best_alpha))

MLP best alpha : 1.0


In [14]:
MLP_best = MLPClassifier(solver='lbfgs',alpha=best_alpha,hidden_layer_sizes=hidden_layer_config)
MLP_best.fit(X_train, Y_train);
Y_pred = MLP_best.predict(X_test)
print( "MLP best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

MLP best accuracy : 0.917
